In [1]:
import torch as t
import itertools as it
import polars as pl
import altair as alt
from sae_lens import SAE
from transformer_lens import HookedTransformer
from datasets import load_dataset
from torch.utils.data import DataLoader
from collections import defaultdict
from tqdm import tqdm


/home/juan/.asdf/installs/python/3.11.9/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = "cuda"
dataset_name = "stas/openwebtext-10k"
transformer_model_name = "gpt2-small"
sae_model_name = "gpt2-small-res-jb"

chunk_size = 20
batch_size = 16

In [3]:
model = HookedTransformer.from_pretrained(transformer_model_name, device=device)

for params in model.parameters():
    params.requires_grad_(False)

/home/juan/.asdf/installs/python/3.11.9/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Loaded pretrained model gpt2-small into HookedTransformer


In [4]:
dataset = load_dataset(dataset_name, split='train', trust_remote_code=True)
dataset = dataset.to_list()
dataset = [x["text"] for x in dataset]

In [5]:
def chunker(iterable, n):
    for i in range(0, len(iterable), n):
        yield iterable[i:i+n]

dataset_chunked = [
    model.tokenizer.decode(chunk)
    for x in dataset[:3]
    for chunk in chunker(model.tokenizer(x, padding=True, truncation=True)["input_ids"], chunk_size)
]

len(dataset_chunked)

128

In [77]:
dataloader = DataLoader(dataset_chunked, batch_size=batch_size)

results = defaultdict(lambda: [])

for block_idx in range(0, 12):
    sae_block_hook = f"blocks.{block_idx}.hook_resid_pre"

    print(f"Calculating results for block {sae_block_hook}")

    sae, cfg_dict, sparsity = SAE.from_pretrained(
        release=sae_model_name,
        sae_id=sae_block_hook,
        device=device)

    for params in sae.parameters():
        params.requires_grad_(False)

    eos_token_id = model.tokenizer.eos_token_id

    for batch_idx, batch in enumerate(dataloader):
        tokens = model.tokenizer(batch, return_tensors="pt", padding=True, truncation=True)
        token_ids = tokens.input_ids

        logits, cache = model.run_with_cache(token_ids)
        sae_feats = sae.encode(cache[sae_block_hook])
        sae_out = sae.decode(sae_feats)

        n = sae_feats.size(0)

        for i, j in tqdm(it.product(r := range(n), r), total=n*n):
            if i >= j: continue

            for m, n in it.product(r := range(chunk_size), r):
                # A lot of false positives from EOS tokens
                if eos_token_id in token_ids[i, :m+1] or eos_token_id in token_ids[j, :n+1]:
                    continue

                f_i = sae_feats[i, m]
                f_j = sae_feats[j, n]

                x_i = sae_out[i, m]
                x_j = sae_out[j, n]

                norm_feat = (f_i - f_j).norm(2)
                norm_out = (x_i - x_j).norm(2)

                results[block_idx].append(dict(norm_feat=norm_feat, norm_out=norm_out))

    del sae
    t.cuda.empty_cache()


Calculating results for block blocks.0.hook_resid_pre


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 256/256 [00:01<00:00, 193.00it/s]


Calculating results for block blocks.1.hook_resid_pre


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 256/256 [00:01<00:00, 193.52it/s]


Calculating results for block blocks.2.hook_resid_pre


...

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 256/256 [00:01<00:00, 192.43it/s]


Calculating results for block blocks.10.hook_resid_pre


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 256/256 [00:01<00:00, 192.24it/s]


Calculating results for block blocks.11.hook_resid_pre


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 256/256 [00:01<00:00, 193.70it/s]


In [79]:
data = pl.DataFrame({
    "layer": [layer for layer in results.keys() for _ in range(len(results[layer]))],
    "norm_feat": [r["norm_feat"] for layer in results.keys() for r in results[layer]],
    "norm_out": [r["norm_out"] for layer in results.keys() for r in results[layer]]
})

data.head()

layer,norm_feat,norm_out
i64,f64,f64
0,7.509549,55.588039
0,7.039531,18.027466
0,6.007368,19.569332
0,6.197616,18.589457
0,6.314634,18.451288


In [32]:
alt.data_transformers.enable("vegafusion")

DataTransformerRegistry.enable('vegafusion')

In [105]:
data_transformed = data.with_columns([
    (pl.col("norm_feat") / pl.col("norm_out")).alias("ratio")
]).filter(pl.col("ratio").is_not_nan())

data_transformed.describe()

statistic,layer,norm_feat,norm_out,ratio
str,f64,f64,f64,f64
"""count""",4.438416e6,4.438416e6,4.438416e6,4.438416e6
"""null_count""",0.0,0.0,0.0,0.0
"""mean""",5.500223,124.023277,356.403407,0.6815
"""std""",3.451945,264.804926,1165.203308,0.208897
"""min""",0.0,0.096759,0.094448,0.03452
"""25%""",3.0,36.046444,48.060841,0.546286
"""50%""",6.0,46.904251,70.235222,0.663186
"""75%""",9.0,63.801945,105.36895,0.819564
"""max""",11.0,1163.995483,20129.162109,2.942404


In [106]:
alt.Chart(data_transformed).mark_bar().encode(
    y=alt.Y("ratio:Q", bin=alt.Bin(nice=True, maxbins=30)),
    x=alt.X("count()", title="count"),
    column=alt.Column("layer:O"),
).properties(
    width=60,
    height=200,
)

alt.Chart(...)